### Example 4.2.1.1: 1-D interpolation on equally spaced input points

In [1]:
#Setting Peridynamic parameters
class PDoperator_:
    def __init__(self, number_input_points = None, xmin = None, xmax = None, 
               number_output_points = None, xvec_org = None, yvec_org = None, 
               n1order = None, nsize = None, order = None):
        self.totnode_org = number_input_points
        self.totnode = number_output_points
        self.xmin = xmin
        self.xmax = xmax
        self.xvec_org = xvec_org 
        self.yvec_org = yvec_org
        self.n1order = n1order
        self.nsize = nsize
        self.order = order
    

In [2]:
#Function that generates grid
class GridGenerator:
    def __init__(init, PDoperator):
        
        xmin = PDoperator.xmin 
        xmax = PDoperator.xmax
        totnode = PDoperator.totnode
        
        xvec = []
        dvolume = []
        dx = (xmax - xmin)/ totnode
        for i in range(totnode):
            xvec.append(round(xmin + dx*i +dx/2.0,6))
            dvolume.append(round(dx*1*1,6))
        init.xvec = xvec
        init.dvolume = dvolume
    
    

In [3]:
#Function that creates lump volumes
def LumpVolumes(PDoperator, gridgenerator):
    
    totnode_org = PDoperator.totnode_org 
    totnode = PDoperator.totnode 
    xvec_org = PDoperator.xvec_org
    
    xvec = gridgenerator.xvec 
    dvolume = gridgenerator.dvolume
    
    
    dvolumes_org = [0] * totnode_org
    tiny = 1E-7
    rho = [0]*totnode_org
    dvolume_org = [0]*totnode_org
    for i in range(totnode):
        rhosum = 0.0
        for j in range(totnode_org):
            rho[j] = 1.0/((abs(xvec[i] - xvec_org[j]))**3 + tiny)
            rhosum = rhosum + rho[j]
        for j in range(totnode_org):
            dvolume_org[j] = dvolume_org[j] + (rho[j] / rhosum)*dvolume[i]
    rhosum = 0.0;
    for j in range(totnode_org):
        rhosum += dvolume_org[j]
    print("Total volume of output points         = ",totnode*dvolume[0] )
    print("Total volume of computed input points = ", rhosum)
    return dvolume_org

In [4]:
#Calculates size of Peridynamic operator
def getSize1D(n1order):
    nsize = n1order + 1
    return nsize

In [5]:
#Function that sets the order of the Differential Operators
def SetDiffOperators1D(nsize):
    order = list(range(0,nsize))
    return order

In [6]:
class NodeFamily:
    def __init__(self, xsi_order = None, xsivec = None, deltax = None):
        self.xsi_order = xsi_order
        self.xsivec = xsivec
        self.deltax = deltax

In [7]:
import pandas as pd
#Function that finds the node family for each Peridynamic node
def GenerateNodeFamily(xvec_org, xvec):
    
    xsi_order = pd.DataFrame({'node':xvec_org})
    xsivec = pd.DataFrame({'node':xvec_org})
    #xsivec = pd.DataFrame()
    
    #Ordering nodes
    for i in range(len(xvec)):
        aux = []
        for j in range(len(xvec_org)):
                aux.append(math.sqrt((xvec[i]-xvec_org[j])**2))
        xsi_order[str(xvec[i])] = aux
        xsivec[str(xvec[i])] = aux
    
    xsi_order.set_index('node', inplace=True)
    for i in xsi_order:
        xsi_order[str(i)] = xsi_order[str(i)].sort_values().keys()
    xsi_order_2 = pd.DataFrame({'node':xvec_org})
    for i in xsi_order:
        xsi_order_2[str(i)] = list(xsi_order[str(i)].sort_values().keys())
    
    #Calculating deltas (horizon) we are selecting the largest bond
    deltax = []
    xsivec = xsivec.drop(columns="node")
    for i in xsivec:
        deltax.append(max(list(xsivec[str(i)])))
    
    NodeFam = NodeFamily()
    xsi_order_2 = xsi_order_2.drop(columns="node")
    NodeFam.xsi_order = xsi_order_2
    NodeFam.xsivec = xsivec
    NodeFam.deltax = deltax
    
    #aux2 +=np.matmul(aux,np.transpose(aux))*dvolumes_org[node_mem]
    return NodeFam;

In [8]:
#Function that calculates the Peridynamics weights
def calc_weights(NodeFam, xvec):
    tiny = 0.000001
    weights = pd.DataFrame() 

    k=0    
    for i in xvec:
        xsivec_column = NodeFam.xsivec[str(i)].to_numpy()
        aux = []
        for xsi_mag in xsivec_column:
            aux.append((NodeFam.deltax[k]/(xsi_mag+tiny))**2)
        weights[str(i)] = aux
        k = k + 1  
    return weights

In [9]:
class p_operator():
    def __init__(self, p_vec=None, p_matrix = None):
        self.p_vec = p_vec
        self.p_matrix = p_matrix
        

In [10]:
import numpy as np
#Calculate p_operator
def p_operator_1d(x0,nsize,dvolumes_org, weights):
    
    x0_array = x0.to_numpy()
    weights_array = weights.to_numpy()
    aux2 = np.zeros((nsize,nsize))
    node_mem = 0;
    
    for i in x0_array:
        aux = np.zeros((nsize,1))
        for j in range(nsize):
            aux[j] = i**j
            print(aux[j])
          
        #aux2 +=np.matmul(aux,np.transpose(aux))
        #aux2 = aux2*dvolumes_org[node_mem]
        aux2 +=np.matmul(aux,np.transpose(aux))*dvolumes_org[node_mem]*weights[node_mem]
        #aux2 +=np.matmul(aux,np.transpose(aux))
        node_mem = node_mem + 1
    
    p_oper = p_operator()
    p_oper.p_vec = aux.reshape((1, nsize)) 
    p_oper. p_matrix = aux2
    return p_oper

In [11]:
class DiffAmat():
    def __init__(self, p_vec_prism=None, A_prism = None, weights = None):
        self.p_vec_prism = p_vec_prism
        self.A_prism = A_prism
        self.weights = weights 

In [12]:
#Form Diff A Matrix
import math

def FormDiffAmat1D(nsize, xvec_org, xvec, dvolumes_org, NodeFam):
    #An A matrix must be created for each of the Peridynamic nodes
    DiffAmat1D = [[0] * PDoperator.nsize] * PDoperator.nsize #Creating DiffAmat 2D matrix
    xsivec = pd.DataFrame()
    xsivec_mags = pd.DataFrame()
    
    for i in range(len(xvec)):
        aux = []
        for j in range(len(xvec_org)):
                aux.append(xvec_org[j]-xvec[i])
        xsivec[str(xvec[i])] = aux
        
    #weights must be calculated here it is going to be one for each Peridynamic node i.e. 200
    weights = calc_weights(NodeFam, xvec)
   
    A_prism = np.zeros((len(xvec), nsize,nsize ))
    p_vec_prism = np.zeros((len(xvec), nsize))
    j=0
    for i in xvec:     
        p_operator = p_operator_1d(xsivec[str(i)],nsize, dvolumes_org,weights[str(i)])
        p_vec_prism[j,:] = p_operator.p_vec
        A_prism[j,:,:] = p_operator.p_matrix
        j = j +1
    
    A_prism_info = DiffAmat()
    A_prism_info.p_vec_prism = p_vec_prism
    A_prism_info.A_prism = A_prism
    A_prism_info.weights = weights
    return A_prism_info
    

In [13]:
def FormInvDiffAmat1D(DiffAmat1D, nsize):
    a,b,c = DiffAmat1D.shape #get dimensions of DiffAmat1D
    DiffAmatInv1D = np.zeros((a, nsize,nsize ))
    for i in range(a):
        DiffAmatInv1D[i] = np.linalg.inv(DiffAmat1D[i])

    return DiffAmatInv1D

In [14]:
def FormDiffAvec(DiffAmatInv1D,DiffBvec1D):
    a,b,c = DiffAmatInv1D.shape #get dimensions of DiffAmatInv1D
    DiffAvec_prism = np.zeros((a, b))

    for i in range(a): 
        DiffAvec_prism[i,:] = np.matmul(DiffBvec1D,DiffAmatInv1D[i])
    return DiffAvec_prism

In [15]:
import numpy as np
import math
#Function that calculates the 1D pass derivative operator
def Derivative_Operator_1D_PASS(PDoperator, gridgenerator, dvolumes_org):
    
    xvec = gridgenerator.xvec
    n1order = PDoperator.n1order
    PDoperator.nsize = getSize1D(n1order)
    PDoperator.order = SetDiffOperators1D(PDoperator.nsize)
    DiffAmat1D = [[0] * PDoperator.nsize] * PDoperator.nsize #Creating DiffAmat 2D matrix
    DiffAmatInv1D = [[0] * PDoperator.nsize] * PDoperator.nsize #Creating DiffAmatInv 2D matrix
    DiffBvec1D = [0] * PDoperator.nsize
    DiffAvec1D = [0] * PDoperator.nsize
    plist = [0] * PDoperator.nsize
    blist = [0] * PDoperator.nsize
    weight = [0] * PDoperator.nsize
    rcvec = [0] * PDoperator.nsize
    wgt = [0] * PDoperator.totnode_org
    ndfam = [0] * PDoperator.totnode_org
    
   
    nsize = PDoperator.nsize
    
    xvec_org = PDoperator.xvec_org
    #Here is where we should calculate A matrix, inverse of A matrix and B vector; for every Peridynamic node.
    
    
    #Generate node family
    NodeFam = GenerateNodeFamily(xvec_org, xvec)
    #Form A array of A Matrices and p_vec_prism
    DiffAmat1D = FormDiffAmat1D(nsize, xvec_org, xvec, dvolumes_org, NodeFam);    
    #Form Inverse of A Matrices
    DiffAmatInv1D = FormInvDiffAmat1D(DiffAmat1D.A_prism, nsize)
    #Form B vector
    DiffBvec1D = np.array([1,0,0])
    #Form a vector
    DiffAvec = FormDiffAvec(DiffAmatInv1D, DiffBvec1D)
    print(DiffAvec[0])
    print(DiffAmat1D.p_vec_prism[0])
    h = DiffAmat1D.weights['0.0225'].to_numpy()
    print(h[0])
    
    print(h[0]*np.matmul(DiffAvec[0],np.transpose(DiffAmat1D.p_vec_prism[0])))
    #print(DiffAmat1D.weights['0.0225'].to_numpy())
    
    #print("Interpolating for N=2")
    #for i in range(PDoperator.totnode):
        #GenerateNodeFamily_0
        
    
    return DiffAvec
    
    

In [16]:
#The PDoperator class constructor has the following inputs
#PDoperator_name = PDoperator(number_input_points, xmin, xmax, output_points, 
#                             xvec_org, yvec_org)
number_input_points = 10
xmin = 0
xmax = 9
number_output_points = 200
xvec_org = list(range(xmin,xmax+1))
yvec_org = [2, 2, 2, 2, 2, 3.2, 3.2, 3.2, 3.2, 3.2]
n1order = 2

PDoperator = PDoperator_(number_input_points, xmin, xmax, number_output_points, 
                        xvec_org, yvec_org, n1order)

#Generating the grid
gridgenerator = GridGenerator(PDoperator)

#Create lump volumes
dvolumes_org = LumpVolumes(PDoperator, gridgenerator)
#Calculating the 1D derivative operator
h = Derivative_Operator_1D_PASS(PDoperator, gridgenerator,dvolumes_org)



Total volume of output points         =  9.0
Total volume of computed input points =  9.0
[1.]
[-0.0225]
[0.00050625]
[1.]
[0.9775]
[0.95550625]
[1.]
[1.9775]
[3.91050625]
[1.]
[2.9775]
[8.86550625]
[1.]
[3.9775]
[15.82050625]
[1.]
[4.9775]
[24.77550625]
[1.]
[5.9775]
[35.73050625]
[1.]
[6.9775]
[48.68550625]
[1.]
[7.9775]
[63.64050625]
[1.]
[8.9775]
[80.59550625]
[1.]
[-0.0675]
[0.00455625]
[1.]
[0.9325]
[0.86955625]
[1.]
[1.9325]
[3.73455625]
[1.]
[2.9325]
[8.59955625]
[1.]
[3.9325]
[15.46455625]
[1.]
[4.9325]
[24.32955625]
[1.]
[5.9325]
[35.19455625]
[1.]
[6.9325]
[48.05955625]
[1.]
[7.9325]
[62.92455625]
[1.]
[8.9325]
[79.78955625]
[1.]
[-0.1125]
[0.01265625]
[1.]
[0.8875]
[0.78765625]
[1.]
[1.8875]
[3.56265625]
[1.]
[2.8875]
[8.33765625]
[1.]
[3.8875]
[15.11265625]
[1.]
[4.8875]
[23.88765625]
[1.]
[5.8875]
[34.66265625]
[1.]
[6.8875]
[47.43765625]
[1.]
[7.8875]
[62.21265625]
[1.]
[8.8875]
[78.98765625]
[1.]
[-0.1575]
[0.02480625]
[1.]
[0.8425]
[0.70980625]
[1.]
[1.8425]
[3.3948062

[1.73580625]
[1.]
[-0.3175]
[0.10080625]
[1.]
[0.6825]
[0.46580625]
[1.]
[1.6825]
[2.83080625]
[1.]
[2.6825]
[7.19580625]
[1.]
[3.6825]
[13.56080625]
[1.]
[4.6825]
[21.92580625]
[1.]
[5.6825]
[32.29080625]
[1.]
[6.6825]
[44.65580625]
[1.]
[-2.3625]
[5.58140625]
[1.]
[-1.3625]
[1.85640625]
[1.]
[-0.3625]
[0.13140625]
[1.]
[0.6375]
[0.40640625]
[1.]
[1.6375]
[2.68140625]
[1.]
[2.6375]
[6.95640625]
[1.]
[3.6375]
[13.23140625]
[1.]
[4.6375]
[21.50640625]
[1.]
[5.6375]
[31.78140625]
[1.]
[6.6375]
[44.05640625]
[1.]
[-2.4075]
[5.79605625]
[1.]
[-1.4075]
[1.98105625]
[1.]
[-0.4075]
[0.16605625]
[1.]
[0.5925]
[0.35105625]
[1.]
[1.5925]
[2.53605625]
[1.]
[2.5925]
[6.72105625]
[1.]
[3.5925]
[12.90605625]
[1.]
[4.5925]
[21.09105625]
[1.]
[5.5925]
[31.27605625]
[1.]
[6.5925]
[43.46105625]
[1.]
[-2.4525]
[6.01475625]
[1.]
[-1.4525]
[2.10975625]
[1.]
[-0.4525]
[0.20475625]
[1.]
[0.5475]
[0.29975625]
[1.]
[1.5475]
[2.39475625]
[1.]
[2.5475]
[6.48975625]
[1.]
[3.5475]
[12.58475625]
[1.]
[4.5475]
[20.6

[1.]
[1.4325]
[2.05205625]
[1.]
[2.4325]
[5.91705625]
[1.]
[3.4325]
[11.78205625]
[1.]
[4.4325]
[19.64705625]
[1.]
[-4.6125]
[21.27515625]
[1.]
[-3.6125]
[13.05015625]
[1.]
[-2.6125]
[6.82515625]
[1.]
[-1.6125]
[2.60015625]
[1.]
[-0.6125]
[0.37515625]
[1.]
[0.3875]
[0.15015625]
[1.]
[1.3875]
[1.92515625]
[1.]
[2.3875]
[5.70015625]
[1.]
[3.3875]
[11.47515625]
[1.]
[4.3875]
[19.25015625]
[1.]
[-4.6575]
[21.69230625]
[1.]
[-3.6575]
[13.37730625]
[1.]
[-2.6575]
[7.06230625]
[1.]
[-1.6575]
[2.74730625]
[1.]
[-0.6575]
[0.43230625]
[1.]
[0.3425]
[0.11730625]
[1.]
[1.3425]
[1.80230625]
[1.]
[2.3425]
[5.48730625]
[1.]
[3.3425]
[11.17230625]
[1.]
[4.3425]
[18.85730625]
[1.]
[-4.7025]
[22.11350625]
[1.]
[-3.7025]
[13.70850625]
[1.]
[-2.7025]
[7.30350625]
[1.]
[-1.7025]
[2.89850625]
[1.]
[-0.7025]
[0.49350625]
[1.]
[0.2975]
[0.08850625]
[1.]
[1.2975]
[1.68350625]
[1.]
[2.2975]
[5.27850625]
[1.]
[3.2975]
[10.87350625]
[1.]
[4.2975]
[18.46850625]
[1.]
[-4.7475]
[22.53875625]
[1.]
[-3.7475]
[14.04375

[0.2275]
[0.05175625]
[1.]
[1.2275]
[1.50675625]
[1.]
[2.2275]
[4.96175625]
[1.]
[-6.8175]
[46.47830625]
[1.]
[-5.8175]
[33.84330625]
[1.]
[-4.8175]
[23.20830625]
[1.]
[-3.8175]
[14.57330625]
[1.]
[-2.8175]
[7.93830625]
[1.]
[-1.8175]
[3.30330625]
[1.]
[-0.8175]
[0.66830625]
[1.]
[0.1825]
[0.03330625]
[1.]
[1.1825]
[1.39830625]
[1.]
[2.1825]
[4.76330625]
[1.]
[-6.8625]
[47.09390625]
[1.]
[-5.8625]
[34.36890625]
[1.]
[-4.8625]
[23.64390625]
[1.]
[-3.8625]
[14.91890625]
[1.]
[-2.8625]
[8.19390625]
[1.]
[-1.8625]
[3.46890625]
[1.]
[-0.8625]
[0.74390625]
[1.]
[0.1375]
[0.01890625]
[1.]
[1.1375]
[1.29390625]
[1.]
[2.1375]
[4.56890625]
[1.]
[-6.9075]
[47.71355625]
[1.]
[-5.9075]
[34.89855625]
[1.]
[-4.9075]
[24.08355625]
[1.]
[-3.9075]
[15.26855625]
[1.]
[-2.9075]
[8.45355625]
[1.]
[-1.9075]
[3.63855625]
[1.]
[-0.9075]
[0.82355625]
[1.]
[0.0925]
[0.00855625]
[1.]
[1.0925]
[1.19355625]
[1.]
[2.0925]
[4.37855625]
[1.]
[-6.9525]
[48.33725625]
[1.]
[-5.9525]
[35.43225625]
[1.]
[-4.9525]
[24.5272

[ 1.53953155e-05  1.40870043e-04 -2.39521389e-05]
[ 1.          8.9775     80.59550625]
159186.8497433574
-103.53172465725382


In [17]:
#np.zeros((len(gridgenerator.xvec), PDoperator.nsize))

In [18]:
#h.to_excel("output.xlsx")

In [19]:

DiffBvec1D = np.array([1,0,0]) 
print(h)
#print(np.matmul(DiffBvec1D,h[1]))

[[ 1.53953155e-05  1.40870043e-04 -2.39521389e-05]
 [ 1.30046865e-04  3.38681164e-04 -6.13820287e-05]
 [ 3.37207334e-04  4.30832557e-04 -8.50883841e-05]
 [ 6.13076981e-04  4.26049408e-04 -9.55259078e-05]
 [ 9.33279342e-04  3.34667669e-04 -9.33782937e-05]
 [ 1.27314917e-03  1.68840404e-04 -7.96072075e-05]
 [ 1.60819334e-03 -5.73118082e-05 -5.55004483e-05]
 [ 1.91476065e-03 -3.27654476e-04 -2.27129939e-05]
 [ 2.17094324e-03 -6.24156829e-04  1.67080538e-05]
 [ 2.35770353e-03 -9.27239925e-04  6.03260963e-05]
 [ 2.46017476e-03 -1.21641623e-03  1.05358097e-04]
 [ 2.46902023e-03 -1.47125718e-03  1.48769927e-04]
 [ 2.38166573e-03 -1.67267611e-03  1.87431107e-04]
 [ 2.20316084e-03 -1.80444129e-03  2.18326731e-04]
 [ 1.94640430e-03 -1.85474798e-03  2.38810965e-04]
 [ 1.63151479e-03 -1.81760434e-03  2.46871702e-04]
 [ 1.28425451e-03 -1.69375415e-03  2.41364376e-04]
 [ 9.33603003e-04 -1.49089804e-03  2.22169906e-04]
 [ 6.08784327e-04 -1.22308937e-03  1.90240893e-04]
 [ 3.36202134e-04 -9.09348466e-

In [20]:
print(gridgenerator.xvec)
print(xvec_org[2]-gridgenerator.xvec[0])

[0.0225, 0.0675, 0.1125, 0.1575, 0.2025, 0.2475, 0.2925, 0.3375, 0.3825, 0.4275, 0.4725, 0.5175, 0.5625, 0.6075, 0.6525, 0.6975, 0.7425, 0.7875, 0.8325, 0.8775, 0.9225, 0.9675, 1.0125, 1.0575, 1.1025, 1.1475, 1.1925, 1.2375, 1.2825, 1.3275, 1.3725, 1.4175, 1.4625, 1.5075, 1.5525, 1.5975, 1.6425, 1.6875, 1.7325, 1.7775, 1.8225, 1.8675, 1.9125, 1.9575, 2.0025, 2.0475, 2.0925, 2.1375, 2.1825, 2.2275, 2.2725, 2.3175, 2.3625, 2.4075, 2.4525, 2.4975, 2.5425, 2.5875, 2.6325, 2.6775, 2.7225, 2.7675, 2.8125, 2.8575, 2.9025, 2.9475, 2.9925, 3.0375, 3.0825, 3.1275, 3.1725, 3.2175, 3.2625, 3.3075, 3.3525, 3.3975, 3.4425, 3.4875, 3.5325, 3.5775, 3.6225, 3.6675, 3.7125, 3.7575, 3.8025, 3.8475, 3.8925, 3.9375, 3.9825, 4.0275, 4.0725, 4.1175, 4.1625, 4.2075, 4.2525, 4.2975, 4.3425, 4.3875, 4.4325, 4.4775, 4.5225, 4.5675, 4.6125, 4.6575, 4.7025, 4.7475, 4.7925, 4.8375, 4.8825, 4.9275, 4.9725, 5.0175, 5.0625, 5.1075, 5.1525, 5.1975, 5.2425, 5.2875, 5.3325, 5.3775, 5.4225, 5.4675, 5.5125, 5.5575, 5.6025,

In [21]:
print('Hello World')

Hello World
